# Clean text for analysis 

## Import modules

In [1]:
from connect_to_mongo import connect_to_mongo
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import porter

## Connect to Mongo database 

In [2]:
db, transcipts = connect_to_mongo('x-files', 'transcripts')

## Explore text entries 

In [3]:
doc_cursor = transcipts.find().limit(1)

In [5]:
for i in doc_cursor:
    scully = i['scully_lines']
    mulder = i['mulder_lines']

In [44]:
scully

['Agent Dana Scully.',
 'Yes, sir.',
 'Well, sir, I was recruited out of medical school. Um, my parents still think it was an act of rebellion, but, uh... I saw the F.B.I. as a place where I could distinguish myself.',
 'Yes, I am.',
 "By reputation. He's an Oxford educated Psychologist, who wrote a monograph on serial killers and the occult, that helped to catch Monty Props in 1988. Generally thought of as the best analyst in the violent crimes section. He had a nickname at the academy... Spooky Mulder.",
 'I believe they have to do with unexplained phenomena.',
 'Am I to understand that you want me to debunk the X-Files project, sir?',
 "Agent Mulder. I'm Dana Scully, I've been assigned to work with you.",
 "Actually, I'm looking forward to working with you. I've heard a lot about you.",
 'If you have any doubt about my qualifications or credentials, th...',
 'Did you bother to read it?',
 'Needle punctures, maybe. An animal bite. Electrocution of some kind.',
 "It's organic. I don't

In [13]:
scully_concat = (' ').join(scully)

In [35]:
scully_concat

'Agent Dana Scully. Yes, sir. Well, sir, I was recruited out of medical school. Um, my parents still think it was an act of rebellion, but, uh... I saw the F.B.I. as a place where I could distinguish myself. Yes, I am. By reputation. He\'s an Oxford educated Psychologist, who wrote a monograph on serial killers and the occult, that helped to catch Monty Props in 1988. Generally thought of as the best analyst in the violent crimes section. He had a nickname at the academy... Spooky Mulder. I believe they have to do with unexplained phenomena. Am I to understand that you want me to debunk the X-Files project, sir? Agent Mulder. I\'m Dana Scully, I\'ve been assigned to work with you. Actually, I\'m looking forward to working with you. I\'ve heard a lot about you. If you have any doubt about my qualifications or credentials, th... Did you bother to read it? Needle punctures, maybe. An animal bite. Electrocution of some kind. It\'s organic. I don\'t know, is it some kind of synthetic protei

In [46]:
mulder_concat = (" ").join(mulder)

#### Quick text cleaning to explore basic word counts

#### Count vectorize 

In [47]:
cv = CountVectorizer(lowercase=True, stop_words='english')
cv_data = cv.fit_transform([scully_concat])

In [48]:
scully_count_vect = pd.DataFrame(cv_data.toarray(), columns=cv.get_feature_names())

In [49]:
scully_count_vect

,1988,academy,accident,act,actually,adolescent,advance,afraid,agenda,agent,...,watch,weighing,wheelchair,woods,work,working,wrote,years,yes,yesterday
0,1,1,1,1,1,1,1,1,1,3,...,1,1,1,4,1,1,1,1,5,1


In [57]:
scully_count_vect.sort_values(by=0, axis=1, ascending=False)

,mulder,know,ve,don,going,kind,yes,did,body,sir,...,heard,helped,hi,hiding,high,honestly,hoped,house,human,yesterday
0,10,9,8,8,6,6,5,5,5,5,...,1,1,1,1,1,1,1,1,1,1


In [58]:
cv = CountVectorizer(lowercase=True, stop_words='english')
cv_data = cv.fit_transform([mulder_concat])
mulder_count_vect = pd.DataFrame(cv_data.toarray(), columns=cv.get_feature_names())
mulder_count_vect

,abducted,abduction,able,academy,access,actually,agenda,ah,alien,allowed,...,wonder,wondering,work,worked,write,ya,yeah,year,yes,zipcode
0,1,2,3,1,1,1,1,1,3,1,...,1,1,2,1,1,1,5,1,2,1


In [62]:
list(mulder_count_vect.columns)

['abducted',
 'abduction',
 'able',
 'academy',
 'access',
 'actually',
 'agenda',
 'ah',
 'alien',
 'allowed',
 'amazing',
 'answers',
 'apart',
 'applying',
 'aptitude',
 'arrange',
 'arranged',
 'attempts',
 'authorities',
 'autopsy',
 'aware',
 'beats',
 'bedpan',
 'behavioural',
 'better',
 'big',
 'billy',
 'bites',
 'blocking',
 'blood',
 'bodies',
 'body',
 'bound',
 'boy',
 'boys',
 'brakes',
 'bright',
 'bureau',
 'buried',
 'called',
 'calls',
 'came',
 'campfire',
 'care',
 'case',
 'cases',
 'cause',
 'causing',
 'cemetery',
 'changed',
 'chemistry',
 'circumstances',
 'city',
 'class',
 'classified',
 'clock',
 'close',
 'closely',
 'come',
 'committed',
 'comprehend',
 'confessed',
 'confirm',
 'congress',
 'connection',
 'connections',
 'continue',
 'control',
 'controls',
 'convention',
 'corpse',
 'couldn',
 'county',
 'coventional',
 'crazy',
 'crime',
 'crimes',
 'criminal',
 'cursory',
 'damn',
 'day',
 'death',
 'deaths',
 'deep',
 'degree',
 'dell',
 'destroy',
 

In [59]:
mulder_count_vect.sort_values(by=0, axis=1, ascending=False)

,know,scully,ve,think,billy,just,did,oh,night,yeah,...,hadn,guy,great,graves,gravedigging,grave,graduating,gotten,gotta,zipcode
0,16,11,11,8,7,6,6,6,5,5,...,1,1,1,1,1,1,1,1,1,1


## Top words for Scully and Mulder over the whole show 

Use a counter, then check if the values are greater than the current top 10....idk. Or change to tuples and then get a sorted tuple list based on the count value (the second value) - quicksort. 

## Adding labels to documents for classification - testing if Scully or Mulder said it 

Make a function that gets the length of the scully or mulder list and creates an array with the data labels for each record. 

In [71]:
import numpy as np

label_dict = {'Scully': 0, 'Mulder': 1}

def get_labels(list_of_docs, character):
    number_of_docs = len(list_of_docs)
    labels = np.array([label_dict[character]] * number_of_docs)
    return labels

In [72]:
scully_list = [scully_concat]

In [73]:
scully_list.append('test')

In [74]:
get_labels(scully_list, 'Scully')

array([0, 0])

### Remove punctuation 

Replace punctuation with a whitespace, for easier removal of stopwords later like I've (stopwords that appear as separate entries in the corpus - for I've it appears as 'i' and 've' in the stopword corpus). 

In [65]:
import re
import string
punc_removed = re.sub('[%s]' % re.escape(string.punctuation), ' ', scully_concat)
# remove extra whitespaces
# regex for 2 or more whitespaces, replace with 1 whitespace

In [64]:
punc_removed

'Agent Dana Scully  Yes  sir  Well  sir  I was recruited out of medical school  Um  my parents still think it was an act of rebellion  but  uh    I saw the F B I  as a place where I could distinguish myself  Yes  I am  By reputation  He s an Oxford educated Psychologist  who wrote a monograph on serial killers and the occult  that helped to catch Monty Props in 1988  Generally thought of as the best analyst in the violent crimes section  He had a nickname at the academy    Spooky Mulder  I believe they have to do with unexplained phenomena  Am I to understand that you want me to debunk the X Files project  sir  Agent Mulder  I m Dana Scully  I ve been assigned to work with you  Actually  I m looking forward to working with you  I ve heard a lot about you  If you have any doubt about my qualifications or credentials  th    Did you bother to read it  Needle punctures  maybe  An animal bite  Electrocution of some kind  It s organic  I don t know  is it some kind of synthetic protein  Do y

### Remove stopwords 

In [61]:
from nltk.corpus import stopwords
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

## Immediate next step:
* Remove extra whitespaces in punctuation section

## To dos: 
* How to label the records as Scully or Mulder - add get_labels into pipeline
* Concatenate all the episodes for Scully, and get word counts for all of them
* Concatenate all the episodes for Mulder's lines, and get word counts for all of them
* Get count vectorized data per line for Scully and Mulder
* Clean the text
* Special words to deal with: 
    * F.B.I. (changes to F B I after replacing punctuation)
    * Names (Mulder says 'Scully' a lot, vice versa)